
<div class="markdown-google-sans">
<h2>Pinecone-PDF</h2>
</div>

**Building an Interactive Chatbot with Langchain, ChatGPT, Pinecone, and Streamlit**

creating a highly efficient chatbot that can answer queries from its own documents or knowledge base. This chatbot, leveraging the power of advanced language models, can also respond to follow-up questions from the users, ensuring a seamless and interactive user experience.

<div class="markdown-google-sans">

## **Install Dependencies**
</div>



In [ ]:
pip install -r requirements.txt

<div class="markdown-google-sans">

## **Train Dataset**
</div>

1.   Data Collection

> The project began with collecting a PDF file containing frequently asked questions and their corresponding answers. This PDF file served as the knowledge base for the chatbot.

> store PDF files in `data` folder


<div class="markdown-google-sans">

## **Training the Model**
</div>


1.   **Loading documents from a directory with LangChain**
> The first step in the `app.py` script involves loading the documents from a directory. We use the `DirectoryLoader` class provided by LangChain to achieve this. This class accepts a directory as input and loads all the documents present in it.



In [ ]:
from langchain.document_loaders import DirectoryLoader

directory = './data'

def load_docs(directory):
  loader = DirectoryLoader(directory)
  documents = loader.load()
  return documents

documents = load_docs(directory)
len(documents)



2.   **Splitting documents**
> After loading the documents, the script proceeds to split these documents into smaller chunks. The size of the chunks and the overlap between these chunks can be defined by the user. This is done to ensure that the size of the documents is manageable and that no relevant information is missed out due to the splitting. The RecursiveCharacterTextSplitter class from LangChain is used for this purpose




In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents,chunk_size=500,chunk_overlap=20):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

docs = split_docs(documents)
print(len(docs))


3.   **Creating embeddings**

>Once the documents are split, we need to convert these chunks of text into a format that our AI model can understand. This is done by creating embeddings of the text using SentenceTransformerEmbeddings class provided by LangChain.

In [ ]:
from langchain.embeddings import SentenceTransformerEmbeddings
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")



4.   **Storing embeddings in Pinecone**

>After the embeddings are created, they need to be stored in a place from where they can be easily accessed and searched. Pinecone is a vector database service that is perfect for this task. The embeddings are stored in Pinecone using the Pinecone class from LangChain.


In [ ]:
import pinecone
from langchain.vectorstores import Pinecone
pinecone.init(
    api_key="",  # find at app.pinecone.io
    environment="gcp-starter"  # next to api key in console
)
index_name = "faq"
index = Pinecone.from_documents(docs, embeddings, index_name=index_name)

This completes the process of document indexing, and we are now ready to move to the main application of our chatbot.

<div class="markdown-google-sans">

## **Building the Chatbot Application with Streamlit**
</div>



1.   With the indexed documents in place, the main part of our task is to build the chatbot application itself. We use Streamlit to create a seamless interactive interface for the chatbot.We accomplish this using the `main.py` file.



In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
    MessagesPlaceholder
)
import streamlit as st
from streamlit_chat import message
from utils import *

st.subheader("Chatbot with Langchain, ChatGPT, Pinecone, and Streamlit")

if 'responses' not in st.session_state:
    st.session_state['responses'] = ["How can I assist you?"]

if 'requests' not in st.session_state:
    st.session_state['requests'] = []

if 'buffer_memory' not in st.session_state:
            st.session_state.buffer_memory=ConversationBufferWindowMemory(k=3,return_messages=True)


system_msg_template = SystemMessagePromptTemplate.from_template(template="""Answer the question as truthfully as possible using the provided context,
and if the answer is not contained within the text below, say 'I don't know'""")


human_msg_template = HumanMessagePromptTemplate.from_template(template="{input}")

prompt_template = ChatPromptTemplate.from_messages([system_msg_template, MessagesPlaceholder(variable_name="history"), human_msg_template])




*   **Session State Initialisation**: Firstly, we initialise two lists 'responses' and 'requests' within Streamlit's session state. These lists store the history of bot responses and user requests respectively.
*   **ConversationBufferWindowMemory**: This memory structure is instantiated with a size of 3, meaning that our chatbot would remember the last three interactions, keeping a manageable memory size for efficiency.
*   **PromptTemplate Construction**: We construct a PromptTemplate for our chatbot. The template contains instructions to the language model (LLM), providing structure and context to the input for the LLM to generate a response. Langchain provides different types of MessagePromptTemplate, which includes AIMessagePromptTemplate, SystemMessagePromptTemplate, and HumanMessagePromptTemplate, for creating different types of messages.



2.   Creating the User Interface
>The Streamlit library allows us to quickly build a user-friendly interface for our chatbot application. The `st.title` function is used to display the chatbot's title at the top of the interface. The user's queries and the chatbot's responses are displayed in a conversation format using the `st.container` and `st.text_input` functions.



In [ ]:
st.title("Langchain Chatbot")
...
response_container = st.container()
textcontainer = st.container()
...
with textcontainer:
    query = st.text_input("Query: ", key="input")
    ...
with response_container:
    if st.session_state['responses']:
        for i in range(len(st.session_state['responses'])):
            message(st.session_state['responses'][i],key=str(i))
            if i < len(st.session_state['requests']):
                message(st.session_state["requests"][i], is_user=True,key=str(i)+ '_user')




3.   **Initializing the Language Model and Conversation**




In [ ]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key="")
...
conversation = ConversationChain(memory=st.session_state.buffer_memory, prompt=prompt_template, llm=llm, verbose=True)


4.   **Generating Responses**
>When the user inputs a query, the chatbot uses the `predict` method to generate a response. The response is then displayed in the chat interface.

In [ ]:
if query:
    with st.spinner("typing..."):
        ...
        response = conversation.predict(input=f"Context:\n {context} \n\n Query:\n{query}")
    st.session_state.requests.append(query)
    st.session_state.responses.append(response)



<div class="markdown-google-sans">

## **Refining Queries and Finding Matches with Utility Functions**
</div>

Once our chatbot is operational, we need to ensure that it can effectively process user queries and find relevant responses. This is achieved through a set of utility functions defined in `utils.py`. Here, we describe the purpose of these functions and their roles in the application.


1.   **Refining Queries with OpenAI**
>The `query_refiner` function is used to take the user's query and refine it to ensure it's optimal for providing a relevant answer. It uses OpenAI's DaVinci model to refine the query based on the current conversation log.


In [ ]:
def query_refiner(conversation, query):
    response = openai.Completion.create(
    model="text-davinci-003",
    prompt=f"Given the following user query and conversation log, formulate a question that would be the most relevant to provide the user with an answer from a knowledge base.\n\nCONVERSATION LOG: \n{conversation}\n\nQuery: {query}\n\nRefined Query:",
    temperature=0.7,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
    )
    return response['choices'][0]['text']



2.   **Finding Matches in Pinecone Index**
>The find_match function is used to find the most relevant documents that match the user's query. It uses the Pinecone vector index to find matches and returns the most relevant text.





In [ ]:
def find_match(input):
    input_em = model.encode(input).tolist()
    result = index.query(input_em, top_k=2, includeMetadata=True)
    return result['matches'][0]['metadata']['text']+"\n"+result['matches'][1]['metadata']['text']



3.   **Tracking the Conversation**
>The get_conversation_string function is used to keep track of the ongoing conversation. It generates a string of the conversation log, including both the user's queries and the chatbot's responses.

In [ ]:
def get_conversation_string():
    conversation_string = ""
    for i in range(len(st.session_state['responses'])-1):
        conversation_string += "Human: "+st.session_state['requests'][i] + "\n"
        conversation_string += "Bot: "+ st.session_state['responses'][i+1] + "\n"
    return conversation_string

>With these utility functions, the chatbot can not only generate responses but also refine the user's queries and find the most relevant answers. This ensures a more effective and user-friendly chatbot experience.

<div class="markdown-google-sans">

## **Example**
</div>

1.   Which Languages supported?
>The supported languages are Hindi and English. Tamil, Telugu, Malayalam, Kannada, Gujarati, Marathi, and Bengali will be available soon.
2.   Does it supported in Desktop Offfline mode?
>Yes, it is supported in Desktop Offline mode for certain use cases like ATM Machines.


<div class="markdown-google-sans">

## **Implementation Demo link**
</div>
  
1.   link: http://122.169.118.18:8501/

